# Анализ данных retail_store_sales

## 1. Загрузка и предварительная обработка данных

### 1.1. Загрузка и вывод схемы
Загрузите файл retail_store_sales.csv.  Выведите первые 5 строк загруженного DataFrame и его схему (df.printSchema()).

In [22]:
df = pd.read_csv("retail_store_sales.csv")
print("Первые строки:")
display(df.head(5))

Первые строки:


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


### 1.2. Очистка названий столбцов
Преобразуйте названия всех столбцов к единому регистру - snake_case.  Выведите обновленную схему DataFrame  или названия столбцов, чтобы убедиться в изменении названий.

In [23]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
print("\nНазвания столбцов после преобразования:")
print(df.columns.tolist())


Названия столбцов после преобразования:
['transaction_id', 'customer_id', 'category', 'item', 'price_per_unit', 'quantity', 'total_spent', 'payment_method', 'location', 'transaction_date', 'discount_applied']


### 1.3. Преобразование типов данных
Проанализируйте к каким типам данных относятся данные в столбцах и приведите столбец к соответствующему типу. Убедитесь, что некорректные или отсутствующие значения преобразуются в null в соответствующих типах данных.

In [24]:
df["transaction_id"] = df["transaction_id"].astype(str)
df["customer_id"] = df["customer_id"].astype(str)
df["category"] = df["category"].astype("string")
df["item"] = df["item"].astype("string")
df["price_per_unit"] = pd.to_numeric(df["price_per_unit"], errors="coerce")
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")
df["total_spent"] = pd.to_numeric(df["total_spent"], errors="coerce")
df["payment_method"] = df["payment_method"].astype("string")
df["location"] = df["location"].astype("string")
df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors="coerce")
df["discount_applied"] = df["discount_applied"].map(
    {"True": True, "False": False, "None": None}
)

print("\nПроверка типов:")
print(df.dtypes)


Проверка типов:
transaction_id              object
customer_id                 object
category            string[python]
item                string[python]
price_per_unit             float64
quantity                   float64
total_spent                float64
payment_method      string[python]
location            string[python]
transaction_date    datetime64[ns]
discount_applied            object
dtype: object


## 2. Очистка и валидация данных

### 2.1. Восстановление отсутствующих item
Так как данные статические для каждого товара, то  составьте справочник товаров в отдельный DataFrame с Category, Item и Rrice Rer Unit.
Для транзакций, где отсутствует название товара , но имеется категория и цена , попытайтесь определить название товара, путём объединения (join) с загруженным справочником товаров. Выведите 20 строк, демонстрирующих восстановленные значения.

In [25]:
catalog = (
    df.dropna(subset=["category", "price_per_unit", "item"])
      .groupby(["category", "price_per_unit"])["item"]
      .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0])
      .reset_index()
      .rename(columns={"item": "item_ref"})
)

# присоединяем и заполняем пропуски item
before_missing = df["item"].isna().sum()
df = df.merge(catalog, on=["category", "price_per_unit"], how="left")
df["item"] = df["item"].fillna(df["item_ref"])
df = df.drop(columns=["item_ref"])
after_missing = df["item"].isna().sum()

print(f"ITEM: пропусков было {before_missing}, стало {after_missing}")
display(df[df["item"].notna()].head(20))

ITEM: пропусков было 1213, стало 609


,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,NaN
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,NaN
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,NaN
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,NaN
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,2023-06-10,NaN
7,TXN_1372952,CUST_21,Furniture,Item_20_FUR,33.5,NaN,NaN,Digital Wallet,In-store,2024-04-02,NaN
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-store,2023-04-26,NaN
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3.0,109.5,Cash,Online,2024-03-14,NaN
10,TXN_8776416,CUST_22,Butchers,Item_3_BUT,8.0,9.0,72.0,Cash,In-store,2024-12-14,NaN


### 2.2. Восстановление Total Spent
Найдите все транзакции, с пропусками в общей сумме и обновите ее, пересчитав её как quantity * price_per_unit для всех записей.

In [26]:
mask_ts = df["total_spent"].isna() & df["quantity"].notna() & df["price_per_unit"].notna()
df.loc[mask_ts, "total_spent"] = df.loc[mask_ts, "quantity"] * df.loc[mask_ts, "price_per_unit"]

print(f"TOTAL_SPENT восстановлено строк: {mask_ts.sum()}")
display(df.loc[mask_ts, ["quantity", "price_per_unit", "total_spent"]].head(20))

TOTAL_SPENT восстановлено строк: 0


,quantity,price_per_unit,total_spent


### 2.3. Заполнение отсутствующих Quantity и Price Per Unit
Для транзакций, где отсутствуют значения о количестве проданного товара , но имеются сумма транзакции и цена за товар , вычислите количество проданного товара и заполните пропущенные значения. Результат приведите к целому числу. 
Аналогично, если  отсутствует цена за единицу товара , но общая сумма и количество имеются, вычислите цену за единицу и заполните пропущенные значения. Округлите до двух знаков после запятой. Выведите 20 строк, демонстрирующих заполненные значения.

In [27]:
# Восстановить quantity (округляем до целого, тип Int64 — допускает NaN)
mask_q = df["quantity"].isna() & df["total_spent"].notna() & df["price_per_unit"].notna()
df.loc[mask_q, "quantity"] = (df.loc[mask_q, "total_spent"] / df.loc[mask_q, "price_per_unit"]).round().astype("Int64")
print(f"QUANTITY восстановлено строк: {mask_q.sum()}")

# Восстановить price_per_unit (2 знака)
mask_p = df["price_per_unit"].isna() & df["total_spent"].notna() & df["quantity"].notna() & (df["quantity"] != 0)
df.loc[mask_p, "price_per_unit"] = (df.loc[mask_p, "total_spent"] / df.loc[mask_p, "quantity"]).round(2)
print(f"PRICE_PER_UNIT восстановлено строк: {mask_p.sum()}")

# Показать первые 20 строк, где что-то из этого было восстановлено
changed = df.loc[mask_q | mask_p, ["category", "item", "quantity", "price_per_unit", "total_spent"]]
display(changed.head(20))


QUANTITY восстановлено строк: 0
PRICE_PER_UNIT восстановлено строк: 609


,category,item,quantity,price_per_unit,total_spent
5,Patisserie,<NA>,10.0,20.0,200.0
11,Milk Products,<NA>,8.0,6.5,52.0
17,Milk Products,<NA>,10.0,27.5,275.0
21,Milk Products,<NA>,3.0,35.0,105.0
32,Food,<NA>,8.0,24.5,196.0
127,Butchers,<NA>,10.0,27.5,275.0
159,Butchers,<NA>,9.0,14.0,126.0
216,Patisserie,<NA>,8.0,5.0,40.0
247,Electric household essentials,<NA>,7.0,21.5,150.5
271,Milk Products,<NA>,9.0,11.0,99.0


### 2.4. Удаление оставшихся строк с пропусками
Удалите оставшийся строки с пропусками в Category, Quantity ,Total Spent и Rrice Rer Unit

In [28]:
before_len = len(df)
df = df.dropna(subset=["category", "quantity", "total_spent", "price_per_unit"])
after_len = len(df)
print(f"Удалено строк с критичными пропусками: {before_len - after_len}")

Удалено строк с критичными пропусками: 604


## 3. Разведочный анализ данных

### 3.1. Самые популярные категории товаров
Рассчитайте общее количество проданных единиц товара  для каждой категории. Определите Топ-5 категорий по общему количеству проданных единиц.

In [29]:
top5 = (
    df.groupby("category", dropna=False)["quantity"]
      .sum()
      .sort_values(ascending=False)
      .head(5)
)
display(top5)

category
Furniture                        8462.0
Food                             8387.0
Beverages                        8358.0
Milk Products                    8339.0
Electric household essentials    8309.0
Name: quantity, dtype: float64

### 3.2. Анализ среднего чека
Рассчитайте среднее значение Total Spent для каждого метода оплаты. Округлите до двух знаков после запятой.
Рассчитайте среднее значение Total Spent для каждой места где прошла оплата. Округлите до двух знаков после запятой.

In [30]:
avg_by_payment = df.groupby("payment_method")["total_spent"].mean().round(2).sort_values(ascending=False)
avg_by_location = df.groupby("location")["total_spent"].mean().round(2).sort_values(ascending=False)

print("Средний чек по способам оплаты:")
display(avg_by_payment)
print("\nСредний чек по локациям:")
display(avg_by_location)

Средний чек по способам оплаты:


payment_method
Cash              131.05
Credit Card       129.13
Digital Wallet    128.72
Name: total_spent, dtype: float64


Средний чек по локациям:


location
Online      130.42
In-store    128.86
Name: total_spent, dtype: float64

## 4. Генерация признаков

### 4.1. Временные признаки
Добавьте два новых столбца на основе Transaction Date:
- day_of_week: День недели
- transaction_month: Месяц транзакции 

In [31]:
df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors="coerce")
df["day_of_week"] = df["transaction_date"].dt.day_name()
df["transaction_month"] = df["transaction_date"].dt.month.astype("Int64")
df[["transaction_date", "day_of_week", "transaction_month"]].head()

,transaction_date,day_of_week,transaction_month
0,2024-04-08,Monday,4
1,2023-07-23,Sunday,7
2,2022-10-05,Wednesday,10
3,2022-05-07,Saturday,5
4,2022-10-02,Sunday,10


### 4.2. Продажи по дням недели
Рассчитайте среднюю сумму продаж (Total Spent) для каждого дня недели. Выведите результаты, отсортированные по дням недели.

In [32]:
order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
avg_by_day = (
    df.groupby("day_of_week")["total_spent"]
      .mean()
      .round(2)
      .reindex(order)
)
display(avg_by_day)

day_of_week
Monday       125.57
Tuesday      129.51
Wednesday    126.82
Thursday     129.28
Friday       134.64
Saturday     131.49
Sunday       130.18
Name: total_spent, dtype: float64

### 4.3. Продажи по месяцам
Рассчитайте среднюю сумму продаж (Total Spent)  для каждого месяца. Выведите результаты, отсортированные по месяцам.

In [33]:
avg_by_month = df.groupby("transaction_month")["total_spent"].mean().round(2).sort_index()
display(avg_by_month)

transaction_month
1     134.69
2     130.66
3     126.83
4     131.81
5     127.40
6     130.95
7     126.57
8     124.28
9     131.45
10    127.85
11    128.79
12    133.15
Name: total_spent, dtype: float64

### 4.4. Признаки клиента (CLV)
Рассчитайте customer_lifetime_value (CLV) для каждого клиента как общую сумму (Total Spent), потраченную этим клиентом за все транзакции. Выведите Топ-10 клиентов по их CLV (customer_id и их CLV).

In [35]:
clv = (
    df.groupby("customer_id")["total_spent"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
display(clv)

customer_id
CUST_24    68452.0
CUST_08    67351.5
CUST_05    66974.5
CUST_16    65570.5
CUST_13    65037.0
CUST_23    64507.0
CUST_10    63155.5
CUST_15    63117.5
CUST_21    62933.0
CUST_02    62046.5
Name: total_spent, dtype: float64